In [1]:
import re
from textblob import TextBlob
import pymongo
import time
import argparse
import json
import string
from dateutil import parser as psr
from datetime import datetime
from config import dbname, dbuser, psswd, host, parameters

connection_string ='mongodb+srv://' + dbuser + ':' + psswd + host + '/' + dbname + "?" + parameters
mongo_client = pymongo.MongoClient(connection_string)

In [2]:
def clean_tweet(tweet): 
    ''' 
    Utility function to clean tweet text by removing links, special characters 
    using simple regex statements. 
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

In [3]:
def get_tweet_sentiment(tweet): 
    ''' 
    Utility function to classify sentiment of passed tweet 
    using textblob's sentiment method 
    '''
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(tweet)
    # set sentiment 
    if analysis.sentiment.polarity > 0: 
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'

In [4]:
from pprint import pprint
with mongo_client:
    db = mongo_client.dbAI
    db_tweets = db.twitter.find({ '#tag': "Greta Thunberg" }, { "text": 1, "_id": 1 }).limit(100)
    tweets = list(db_tweets)
    

for tweet in tweets:
    ctweet = clean_tweet(tweet["text"])
    stweet = get_tweet_sentiment(ctweet)
    pprint("Sentiment {} - Tweet {}".format(stweet,ctweet))

('Sentiment neutral - Tweet RT liberals have decided that the only necessary '
 'measure to stop climate change is to appease the child god Greta with idolat')
('Sentiment positive - Tweet RT didn t she explicitly say she wants change not '
 'idolatry')
('Sentiment positive - Tweet RT didn t she explicitly say she wants change not '
 'idolatry')
('Sentiment positive - Tweet It wont be there for long Carbon neutral '
 'biodegradable sustainable LGBTQ friendly anti Trump spray paint ma')
'Sentiment neutral - Tweet RT Greta Thunberg will not be silenced'
('Sentiment positive - Tweet Greta Thunberg turns down environmental prize '
 'says climate movement does not need any more awards musicnews360')
('Sentiment negative - Tweet RT us People are suffering and dying from the '
 'climate emergency and it will continue to get worse Doesn t this mean '
 'anything to the')
('Sentiment negative - Tweet RT Shrike The Greta Thunberg of the 1980s have I '
 'got this wrong')
('Sentiment positive - Twe

In [5]:
import pandas as pd 
df = pd.read_csv('https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv')

df.to_csv('./data/tweets_labelled.csv')
